# Grab glassdoor links and review amounts

### Load dataframe

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote
import random
import time
from pathlib import Path
from torpy.http.requests import TorRequests

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
input = pd.read_csv(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\0813_UniqueFirmList.csv", sep=";", decimal=",", encoding='unicode_escape')

## yahoo scraper

#### proxy rotation via tor

In [5]:
def yahoo_scraper(df, sleeptime_max=1, tor_active="no"):
    try: #if operation fails, then return the current df
        firmname = df["CompanyName"]
        print(f"Firm #{df.name + 1}: {firmname}")
        # query = f'site:glassdoor.com {firmname}'.replace(' ', '+')
        query = f'site:glassdoor.com "{firmname}"'.replace(' ', '+') #putting {firmname} around "" might help. however, the query might also be too strict not returning any results
        URL = f"https://search.yahoo.com/search?p={query}&vc=en&pz=20"

        ## shuffle through user agent list for bot prevention. use IP shuffling to prevent further bot detection
        time.sleep(round(random.uniform(0.25,sleeptime_max), 3))
        USER_AGENTS = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36 Edg/103.0.1264.71"
                    ]
        headers = {"user-agent":random.choice(USER_AGENTS)}

        ## use tor
        if tor_active=="yes":
            url = sess.get(URL, headers=headers)
        ## use regular IP
        else:
            url = requests.get(URL, headers=headers)

        soup = BeautifulSoup(url.content, "html.parser")
        time.sleep(0.2) #to fully load the page

        ## scraper ##
        #############

        link_found = 0
        ## check if amount of ratings is displayed in at least of the search results. if so, grab this number and the parent div containing the link
        if soup.find("li", {"class":["tc", "bxz-bb"]}) and len(soup.find("li", {"class":["tc", "bxz-bb"]}).text)<15: #check length of the classes text, since class is not exclusively used for amount of ratings
            ## grab the review number
            review_amount = soup.find("li", {"class":["tc", "bxz-bb"]}) #only get the first item, since it appears to be most accurate. could be of the last search result
            try: #sometimes "Currency: EUR" is displayed instead of the review amount
                review_amount_final = review_amount.text.split("(")[1][:-1] #"4/5 (3)"
                if "K" in review_amount_final: # "5.7K" to 5700
                    review_amount_final = review_amount_final.replace(".", "")
                    review_amount_final = review_amount_final.replace("K", "00")
                # print(f"review amount: {review_amount_final}")
                df["review_amount"] = review_amount_final
                ## grab the href from the parent of the same search result
            except:
                df["review_amount"] = "" 
            review_parents = review_amount.find_parents("div")
            for link in review_parents:
                if link.find("a") and "glassdoor" in link.find("a")["href"]:
                    alink = link.find("a")
                    link_found = 1
                    break
        ## if amount of ratings is not on page, then just grab the href
        elif link_found != 1:
            # print("review amount: none found")
            results = soup.findAll("h3")
            for link in results:
                if link.find("a") and "glassdoor" in link.find("a")["href"]:
                    alink = link.find("a")
                    link_found = 1
                    break
        ## if neither link nor review amount was found
            if link_found != 1:
                link_found = "no link found" #"no link found"
                link_final = ""
                print("no link found")

        
        ## decode href link if link was found ##

        if link_found == 1:
            link_with_yahoo = unquote(alink["href"]) #unquote decodes URLs
            try: #weird cases in which an image link is retrieved
                link_final = link_with_yahoo.split("RU=")[1].split("/RK=2")[0] #https://r.search.yahoo.com/_ylt=A0geK9dD9t9i2BIAtodXNyoA;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1658873541/RO=10/RU=https://www.glassdoor.com/Overview/Working-at-California-Micro-Devices-EI_IE1221.11,35.htm/RK=2/RS=Ss7CVR85Jhnt899NkWVSw9eSVUw-
                if "/Working" in link_final:
                    # from https://www.glassdoor.com/Overview/Working-at-Tesla-EI_IE43129.11,16.htm
                    # to https://www.glassdoor.com/Reviews/Tesla-Reviews-E43129.htm
                    link_final = link_final.replace("/Overview/Working-at-", "/Reviews/") #shouldnt be an issue to replace the /Overview/ part with /Reviews/ even though the original Reviews link looks different
                    link_final = link_final.replace("EI_IE", "Reviews-E") 
                    link_final = link_final.split(".htm")[0][:-6] + ".htm"
                    link_found = "transformed overview link"
                elif "/Jobs/" in link_final:
                    link_final = link_final.replace("/Jobs/", "/Reviews/")
                    link_final = link_final.replace("-Jobs-", "-Reviews-")
                    link_found = "transformed jobs link"
                elif "/Salary/" in link_final:
                    link_final = link_final.replace("/Salary/", "/Reviews/")
                    link_final = link_final.replace("-Salaries-", "-Reviews-")
                    link_found = "transformed salary link"
                elif "/Reviews/" in link_final:
                    link_found = "original review link"
                else:
                #elif "/job-listing/" or "/Job/" in link_final:
                    link_found = "wrong link"
            except:
                link_final = link_with_yahoo
                
        df["Link_Hannes"] = link_final
        df["link_found"] = link_found
        print(link_final, "\n")
        
        return df

    except:
        return df

# yahoo_scraper("El Paso Pipeline Partners")

In [6]:
## set parameters
startrow, endrow = 13000, 13500
tor = "yes" # change to yes or no
sleeptime_max = 0.5 #use >2 with original ip (tor = "no"). around 1000 requests possible until temp. ip block


## create a copy of the input frame; keep old index as extra column
input_copy = input[startrow:endrow].reset_index(drop=False)

input_copy["Link_Hannes"] = np.nan
input_copy["review_amount"] = np.nan
input_copy["link_found"] = np.nan

## execute yahoo scraper for each row by using apply ##
#######################################################

## with Tor-Gateway
if tor == "yes":
    with TorRequests() as tor_requests:
        print("Connecting to tor..")
        with tor_requests.get_session() as sess: #sess pulls a request of the page with the tor connection
            current_ip = sess.get("http://httpbin.org/ip").json()
            print(f"Current IP:{current_ip}; sleeptime: {sleeptime_max}\n")
            input_copy = input_copy.apply(yahoo_scraper, args=(sleeptime_max, "yes"), axis=1) #sess method can be executed from this sibling function
## without tor
else:
    input_copy = input_copy.apply(yahoo_scraper, args=(sleeptime_max, "no"), axis=1)
print(f"Scraping finished for {startrow} to {endrow}")
    

## reorder columns
# input_copy.keys() #['index', 'CompanyName', 'Ticker', 'ISIN', 'Link_Marius', 'Max_Cosine_Similarity', '2008_2022', 'EnglishSpeaking', 'Link_Hannes', 'review_amount', 'link_found']
input_copy = input_copy[['index', 'ISIN', 'Ticker', '2008_2022', 'EnglishSpeaking', 'CompanyName', 'Max_Cosine_Similarity', 'Link_Marius', 'Link_Hannes', 'link_found', 'review_amount']]


## save file with dynamic file name
path_with_time = Path.joinpath(Path.cwd().parent, f'{time.strftime("%m%d")}_UniqueFirmList_HannesLinks_{startrow}-{endrow}_strict.csv')
#"W:\019_Glassdoor\1 Data\1 Glassdoor Links\0808_UniqueFirmList_HannesLinks_1500-1509.csv"
input_copy.to_csv(path_with_time, sep=";", decimal=",", index=False)
print(f"{path_with_time} saved")

Connecting to tor..
Current IP:{'origin': '185.100.87.129'}; sleeptime: 0.5

Firm #1: Resouro Gold 


c:\Users\g21728\AppData\Local\Programs\Python\Python37\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #2: Responsys
https://www.glassdoor.com/Reviews/Responsys-Reviews-E9722.htm 

Current IP:{'origin': '185.100.87.129'}
Firm #3: Restance 
https://www.glassdoor.com/Reviews/Resistance-Electrical-Contracting-Reviews-E5821642.htm 

Current IP:{'origin': '185.100.87.129'}
Firm #4: Restorgenex
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #5: Retrospettiva
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #6: Rhype
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #7: Richmond Row Capital
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #8: RM Health International
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #9: RM Investors
https://www.glassdoor.com/Reviews/Standard-Chartered-Bank-Institutional-relationship-manager-Hong-Kong-Reviews-EI_IE226853.0,23_KO24,58_IL.59,68_IN106.htm 

Current IP:{'origin': '185.100.87.129'}
Firm #10: RNY Property Trust
no link found
 

Stream #12: closed (but received b'X\xccl\xa2\x07\xaf\x9dO\xb3)\x94\xe6\x9a\x14\xa9\x93N#\xfd\xf5=Nwy\x87\xa16W\xb9\x89dy\x07RJ%;mk\xbaK\x95\x99\xf4\x87\xe7(\\\xe4\xd4ya[\xb6+\x01\xd1\x077\x8a3\x82\xf2\x18d\x0f\xcd\x9d/U\xf7\xfe\xb9\xc9\x16\xc6\xa6+\x8ed\x88\xc0\xcc\x0b\xdb\xf8|\xe7\xa9M\xf9S\x8b8\xf1\xac\x04`|\xb7\r\x0f7c\x84H\xad;\xc4\x17\x85\xa0\x18\xddR\xc7\x16r\x8b\x9d\x96\x96\x81\x8d$\xdcn4\xc0)\xbb\xc6\xed<\xbf_\xed\xcfH\x02\xaa\xa9\xce\xdb\x1e\x14s\t\x1e!\xd4\xd5\xf2\xd8?W\x9eb\xa8\x14lPa\xa5J\xca\xb1\xc5\xae\x1b^\xc5\xd9B\x1d\xf5Q\xa2HF\xb1\xab\x92\xaf\xc5\xba\xfe\xc5\x82+\x10N\x7f\xf3"l\x93L\x8e%O\x00$l_\xdc\xfe \x84H\xc4\xbc\x19\xe4(\x95{\xb5\x92\x86t>k-c\x96U\x83\xe7\xae\xa9\x06\x1c\xd3.O\x94\x83>\x88\xf4\xacq\x12\xae\xc8,\xd6\xc0\x1bu\xbf\xe4\x1f\x11>\x1a\x87\x9er\xf4`1\x18\'q]\xff\xa63\x0b\xdd\x88\x106\xe25k\x97\xf6\xb2\x91(\xef-\xf6\xa5\x81t\t\x04\xe6`\x11\xa8\xd6\x0bB\x16\xed\xe2\x1e\xdf_M\x8c ]c\x9c\xd9\xf1\x13)\x03g\x0c\xd9\x87`\xe3\xe8Bi\xa42CMhW\x1e\xb2\x82d\xee\xae

Firm #99: Uonlive 


Stream #12 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 12, digest = b'\xb7\x1eK\xe6', circuit_id = 80000005))
Stream #12 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 12, digest = b'"\xa9\xb6\xad', circuit_id = 80000005))
Stream #12 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 12, digest = b'\xe4j#\x85', circuit_id = 80000005))
Stream #12 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 12, digest = b'\x1d\xeaD\x98', circuit_id = 80000005))
Stream #12 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 12, digest = b'\xb4\xd9\xfb\xf6', circuit_id = 80000005))
Stream #12 is already closed or was neve

no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #100: Upper Street Marketing
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #101: V.F. 
https://www.glassdoor.com/Reviews/V-and-F-Transformer-Reviews-E3538589.htm 

Current IP:{'origin': '185.100.87.129'}
Firm #102: V2K International
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #103: ValueSetters
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #104: Vampt America
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #105: Vasogen
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #106: Vaycaychella 
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #107: Velcera
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #108: VeriTeQ 
https://www.glassdoor.com/Reviews/Veriteq-Instruments-Reviews-E255188.htm 

Current IP:{'origin': '185.100.87.129'}
Firm #109: VetaNova 
no link found
 

Current IP:{'origin': '185.100.87.129'}
Firm #110: ViaDerma 
